In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split

# Building the model

In [2]:
# load data and split it into Test, train and validation sets

(X_1, y_1), (X_2, y_2) = tf.keras.datasets.cifar10.load_data()
X = np.vstack((X_1, X_2))
y = np.vstack((y_1, y_2))
X_train, X, y_train, y = train_test_split(X, y, train_size=0.8, stratify=y)
X_test, X_valid, y_test, y_valid = train_test_split(X, y, test_size=0.5, stratify=y)

assert X_test.shape == X_valid.shape, 'X_test and X_valid have different shapes'
for i in range(10):
    assert np.count_nonzero(keras.utils.to_categorical(y_train)[:, i]) == 4800 , 'Sample is not stratified'
sample_shape = X_train[0].shape
print(f'Shape of a single sample: {sample_shape}')

Shape of a single sample: (32, 32, 3)


In [3]:
# Normalize the data

X_train = keras.utils.normalize(X_train)
X_valid = keras.utils.normalize(X_valid)
X_test = keras.utils.normalize(X_test)

In [4]:
# build the model

model = Sequential()
model.add(Input(shape=sample_shape))
model.add(Conv2D(8, kernel_size = (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(12, kernel_size = (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(16, kernel_size = (3,3), padding='same', activation='relu'))
model.add(UpSampling2D())
model.add(Conv2D(12,kernel_size = (3,3), padding='same', activation='relu'))
model.add(UpSampling2D(size = (2,2)))
model.add(Conv2D(3,(3,3), padding='same', activation='relu'))


# compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanAbsoluteError(),
            keras.metrics.CosineSimilarity()])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 8)         224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 12)        876       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 12)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 16)          1744      
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 12)        1

2021-11-04 17:05:41.000542: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# train the model
history = model.fit(X_train, X_train, epochs=10, verbose=1, use_multiprocessing=True)

2021-11-04 17:05:43.340157: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1500/1500 [==============================] - 50s 33ms/step - loss: 0.0071 - mean_absolute_error: 0.0458 - cosine_similarity: 0.9918
Epoch 2/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0022 - mean_absolute_error: 0.0279 - cosine_similarity: 0.9958
Epoch 3/10
1500/1500 [==============================] - 47s 32ms/step - loss: 0.0020 - mean_absolute_error: 0.0256 - cosine_similarity: 0.9962
Epoch 4/10
1500/1500 [==============================] - 49s 32ms/step - loss: 0.0018 - mean_absolute_error: 0.0241 - cosine_similarity: 0.9964
Epoch 5/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.0017 - mean_absolute_error: 0.0232 - cosine_similarity: 0.9965
Epoch 6/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0016 - mean_absolute_error: 0.0225 - cosine_similarity: 0.9966
Epoch 7/10
1500/1500 [==============================] - 47s 31ms/step - loss: 0.0016 - mean_absolute_error: 0.0221 - cosine_similarity: 0.9967

In [8]:
# save trained model

def save_model(model, path='models/CAE'):
    model.save(path)

save_model(model)

INFO:tensorflow:Assets written to: models/CAE/assets


In [7]:
# load a pretrained model

def load_model(path = 'models/CAE'):
    return keras.models.load_model(path)


pretrained_model = load_model()